# Keras
## Introducción
Keras es un API de alto nivel para redes neuronales, escrita en Python que es capaz de correr sobre TensorFlow, CNTK o Theano. Fue desarrollada con énfasis en habilitar experimentación rápida. Es recomendable el uso de Keras cuando se requiere de una libería de Deep Learning que:
- Permita el desarrollo de prototipos fácil y rápido (por medio de facilidad de uso, modularidad y extensibilidad).
- Soporte redes neuronales convolucionales (CNN) y redes neuronales recurrentes (RNN), así como combinaciones de ambas.
- Corra de forma natural en CPUs y GPUs.

Keras es compatible con Python 2.7-3.6.

[keras.io](https://keras.io) es el sitio principal del proyecto.

## Backend Keras y tf.keras
Se recomienda usar `tf.keras` a usuarios que usen Keras multi-backend con TensorFlow en TensorFlow 2.0 ya que `tf.keras` tiene mejor integración con las características de TensorFlow 2.0 (eager execution, soporte a distribución).

## Principios
- **Amigable**. El API está diseñado para personas, es consistente, reduce el número de pasos para casos de uso comunes. 
- **Modular**. Un modelo de deep learning consta de una secuencia o un grafo de módulos configurables que pueden ser conectados  con la menor cantidad de restricciones. Las capas de las redes neuronales, las funciones de costo, los optimizadores, esqueasm de inicialización, funciones de activación y esquemas de regularización son módulos independientes que pueden combinarse para crear nuevos modelos.
- **Extensible**. Es posible añadir módulos nuevos
- **Trabaja con Python**. No requiere de archivos de configuración de modelos. Los modelos se describen en Python, que es compacto, fácil de depurar y facilita la extensibilidad.

## Estructuras de datos
La estructura central de Keras es un modelo, que representa una forma de organizar capas. `Sequential` representa al modelo más simple, una pila linear de capas. El **Functional API**, por medio de `Model` permite crear arquitecturas más complejas por medio de un grafo arbitrario de capas.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Carga y preprocesamiento de datos
Las redes neuronales no procesan datos crudos como archivos de texto, imágenes codificadas en JPEG y otros formatos, o archivos CSV. Procesan representaciones vectorizadas y estandarizadas.
* Los archivos de texto necesitan ser leidos a tensores de sequencias de caracteres, divididos en palabrsa. Las palabras necesitan ser indexadas y convertidas a tensores de enteros.
* Las imágenes necesitan ser leídas y decodificadas en tensores de imágenes, convertidas a punto flotante y normalizados a valores pequeños (generalmente entre 0 y 1).
* Los datos CSV necesitan ser leídos, con características numéricas convertidas a tensores de punto flotante y características categóricas indexadas y convertidas a tensores enteros. Cada característica necesita ser normalizada con media cero y variación unitaria.

In [4]:
#Obtener un dataset etiquetado de imágenes en disco
# Create a dataset.
dataset = keras.preprocessing.image_dataset_from_directory(
  'path/to/main_directory', batch_size=64, image_size=(200, 200))

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/main_directory'

In [5]:
#Obtener un dataset etiquetado de archivos de texto en disco
dataset = keras.preprocessing.text_dataset_from_directory(
  'path/to/main_directory', batch_size=64)

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64,)
   print(data.dtype)  # string
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/main_directory'

## Preprocesamiento de datos en Keras
* Generar tokens de secuencias de caracteres, posteriormente índices de tokens.
* Normalizar características.
* Reescalar los datos a valores pequeños (media cero y variación unitaria o datos en el rango [0, 1]).

Keras permite realizar el preprocesamiento de datos como parte del modelo.

In [6]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Example training data, of dtype `string`.
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="int")

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)

tf.Tensor(
[[4 5 2 9 3]
 [7 6 2 8 3]], shape=(2, 5), dtype=int64)


In [7]:
# Convertir secuencias de caracteres a bigrams de tipo one-hot encoding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Example training data, of dtype `string`.
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="binary", ngrams=2)

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)

tf.Tensor(
[[0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]], shape=(2, 17), dtype=float32)


In [8]:
# Normalización de características
from tensorflow.keras.layers.experimental.preprocessing import Normalization

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))

var: 1.0000
mean: 0.0000


In [9]:
# Cambiar la escala, centrar y recortar imágenes
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

cropper = CenterCrop(height=150, width=150)
scaler = Rescaling(scale=1.0 / 255)

output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))

shape: (64, 150, 150, 3)
min: 0.0
max: 1.0


## Construyendo modelos con el API Funcional de Keras
Una capa es una transformación de entrada/salida
```
dense = keras.layers.Dense(units=16)
```
Un modelo es un grafo acíclico dirigido de capas. 

El API Functional es la forma más frecuente y poderosa de crear modelos con Keras. Para construir modelos con el API Functional, se especifican las dimensiones  y el tipo de datos de los datos de entrada. Es posible especificar dimensiones variables con None. Por ejemplo, una imagen 200x200 RGB tiene la forma (200,200, 3), paro una imagen de dimensiones arbitrarias RGB tendría la forma (None, None, 3).

```python
# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(None, None, 3))
```
Después de definir las entradas, es posible encadenar las transformaciones de las capas después de las entradas hasta la salida final.


In [10]:
from tensorflow.keras import layers

# Center-crop images to 150x150
x = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

NameError: name 'inputs' is not defined

En cuanto se haya definido el grafo de capas que convierte las entradas en salidas, es posible instanciar el objeto Model().
```
model = keras.Model(inputs=inputs, outputs=outputs)
```
Es posible invocar el modelo en subconjuntos de datos

In [12]:
model = keras.Model(inputs=inputs, outputs=outputs)

NameError: name 'inputs' is not defined

In [11]:
data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)

NameError: name 'model' is not defined

Es posible desplegar un resumen de las trasnformaciones de datos en cada etapa. Esto es útil para depuración.
```
model.summary()
```

## Entrenamiento de modelos con fit()
Hasta este punto:
* Los datos se encuentran preparados.
* Se ha definido un modelo para ajustar los datos.
El siguiente paso es entrenar el modelo con los datos. La clase Model cuenta con un ciclo de entrenamiento, el método fit().

Antes de invocar fit() se debe especificar un optimizador y una función de pérdida. Este es el paso compile()


In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

La función de pérdida y el optimizador puede ser especificada por medio de sus identificadores como cadenas de caracteres

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Cuando el modelo está compilado, es posible comenzar a ajustar el modelo a los datos.

In [ ]:
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)

In [ ]:
También es posible especificar el tamaño del batch y el número de épocas (iteraciones sobre los datos).

In [ ]:
model.fit(dataset_of_samples_and_labels, epochs=10)

## Clasificación de dígitos en Keras


Importar datos

In [21]:
import numpy as np
import mnist
import keras

from keras.utils import to_categorical


train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### Preprocesar los datos

In [22]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(test_images.shape)
print(test_labels.shape)

(10000, 784)
(10000,)


### Construir el modelo

`model = Sequential([  #layers...])`

El constructor de Sequential recibe como parámetro un arreglo de [Layers](https://keras.io/layers/about-keras-layers/) de Keras

Para el caso de una red neuronal feed forward solo se requiere de la capa [Dense](https://keras.io/layers/core/#dense), que representa una capa completamente conectada.

En este ejemplo añadiremos tres capas al modelo, las primeras dos con 64 nodos cada unoa con la función de activación [relu](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7) y la última con 10 nodos y función de activación [softmax](https://victorzhou.com/blog/softmax/).

Keras requiere conocer la forma de los datos de entrada por medio del parámetro `input_shape` especificado en la primera capa.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

### Compilar el modelo
Es necesario configurar el proceso de entrenamiento considerando tres factores:
- El *optimizador*.
- La *función de pérdida (loss function)*.
- Las *métricas* de optimización.

In [23]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

### Entrenar el modelo
Para entrenar el modelo se invoca la función `fit` con una serie de [parámetros](https://keras.io/models/sequential/#fit), entre los que destacan:
- *Datos de entrenamiento*, incluyendo datos y etiquetas.
- *Número de épocas (epochs)*, que representa el número de iteraciones sobre el dataset completo.
- *El tamaño del batch* número de muestras por actualización de gradiente.

Para este caso Keras espera los targets de entrenamiento como un vector de 10 dimensiones, dado que hay 10 nodos en la capa de salida Softmax, sin embargo train_labels provee de un solo entero que representa la clase de cada imagen. Es por ello que se hace uso del método [to_categorical](https://keras.io/utils/#to_categorical) que convierte al arreglo de enteros en un arreglo de vectores one-hot., por ejemplo 2 estaría representado como `[0,0,1,0,0,0,0,0,0,0,0]` (indexado en cero).

In [24]:
model.fit(
  train_images, # training data
  to_categorical(train_labels), # training targets
  epochs=5,
  batch_size=32,
)

Epoch 1/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.0917 - accuracy: 0.9705
Epoch 2/5
60000/60000 [==============================] - 6s 92us/step - loss: 0.0825 - accuracy: 0.9740
Epoch 3/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.0715 - accuracy: 0.9776
Epoch 4/5
60000/60000 [==============================] - 6s 94us/step - loss: 0.0689 - accuracy: 0.9778
Epoch 5/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.0625 - accuracy: 0.9801


### Probar el modelo
Keras provee el método [evaluate](https://keras.io/models/sequential/#evaluate) que regresa un arreglo con la pérdida en prueba (test loss) seguida de las métricas especificadas.

In [25]:
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

10000/10000 [==============================] - 1s 52us/step


[0.09751011377617251, 0.970300018787384]

### Usar el modelo
El modelo puede guardarse y recuperarse de disco por medio de las funciones `save_weights` y `load_weights`

La función `predict` de un modelo permite predecir con datos de prueba o instancias nuevas.

In [26]:

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[7 2 1 0 4]
